In [1]:
import sys
sys.path.append('../')
from Proximity import prox_model
import geopandas as gpd
import numpy as np

In [2]:
lu=gpd.read_file('https://gis.cambridgema.gov/download/shp/CDD_LandUse.shp.zip')

Take a subset

In [3]:
lu=lu.cx[760000:, 2.95e6:2.96e6].copy()

In [4]:
lu['Category'].unique()

array(['Residential', 'Mixed Use Residential', 'Office/R&D',
       'Higher Education', 'Charitable/Religious', 'Vacant Commercial',
       'Office', 'Privately-Owned Open Space', 'Vacant Residential',
       'Commercial', 'Education', 'Transportation',
       'Education Residential', 'Assisted Living/Boarding House',
       'Public Open Space', 'Mixed Use Commercial', 'Mixed Use Education',
       'Utility', 'Health', 'Industrial', 'Government Operations',
       'Vacant Industrial'], dtype=object)

In [5]:
lu['Office']=lu.geometry.area * (lu['Category'].str.contains('Office')).astype(int)
lu['Residential']=lu.geometry.area * (lu['Category'].str.contains('Residential')).astype(int)

In [6]:
lu=lu.to_crs('EPSG:4326')

In [7]:
lu['source']=True

In [8]:
target_settings={
    'Jobs':{
        'column': 'Office', 
        'max': 10000,
        'from': 'Residential'
    },
    'Housing':{
        'column': 'Residential', 
        'max': 10000,
        'from': 'Office'
    }}

In [9]:
pm=prox_model(places=lu, max_dist=1200, target_settings=target_settings,
                network_type='walk')

Getting network inside buffered outline from OSM


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=sum([c.x for c in geom.geometry.centroid])/len(geom)
/opt/anaconda3/envs/prox/lib/python3.10/site-packages/osmnet/load.py:445: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  gdf.crs = crs
/opt/anaconda3/envs/prox/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/envs/prox/lib/

Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](42.34217652,-71.15915910,42.38553659,-71.05174073);>;);out;'}"
Downloaded 18,629.1KB from www.overpass-api.de in 4.97 seconds


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/osmnet/load.py:171: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  ' {:,.2f} seconds'.format(len(geometry), time.time()-start_time))


Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 5.28 seconds
Returning OSM data with 105,121 nodes and 27,861 ways...
Edge node pairs completed. Took 12.75 seconds
Returning processed graph with 37,386 nodes and 58,085 edges...
Completed OSM data download and Pandana node and edge table creation in 19.07 seconds
Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 37386
Setting CH edge vector of size 58579
Range graph removed 2084 edges of 117158
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
Finding closest nodes to each place


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=sum([c.x for c in geom.geometry.centroid])/len(geom)


Building distance matrix
Finding closest nodes to each place


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/examples/../Proximity.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=sum([c.x for c in geom.geometry.centroid])/len(geom)


In [10]:
access_df=pm.calculate_access(lu)

In [11]:
heatmap=pm.get_heatmap(lu, access_df)

In [13]:
heatmap.explore(column='Housing')